In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [20]:
def prepare_seq(seq, x2idx):
    idxs = [x2idx[w] for w in seq]
    return Variable(torch.LongTensor(idxs))

In [4]:
train_data = [
    ('The dog ate the apple'.split(), ['DET', 'NN', 'V', 'DET', 'NN']),
    ('Everybody read that book'.split(), ['NN', 'V', 'DET', 'NN'])
]
word2idx = {}
for sent, tags in train_data:
    for word in sent:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

print(word2idx)

tag2idx = {'DET': 0, 'NN': 1, 'V': 2}


{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [5]:
embedding_dim = 6
hidden_dim = 6

In [11]:
class LSTMTagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim)
        self.fc = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # h0's shape: (num_layers*directions, batch_size, hidden_dim)
        return (Variable(torch.zeros(1, 1, self.hidden_dim)),
                Variable(torch.zeros(1, 1, self.hidden_dim)))
    
    def forward(self, sent):
        embedding = self.embedding(sent)
        lstm_out, self.hidden = self.lstm(embedding.view(len(sent), 1, -1), self.hidden)
        fc_out = self.fc(lstm_out.view(len(sent), -1))
        tag_probs = F.softmax(fc_out, dim=1)
        return tag_probs
    
        
        

In [12]:
model = LSTMTagger(embedding_dim, hidden_dim, len(word2idx), len(tag2idx))
loss_fun = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [13]:
inputs = prepare_seq(train_data[0][0], word2idx)
tag_probs = model(inputs)
print(tag_probs)

Variable containing:
 0.2939  0.4778  0.2283
 0.2591  0.5045  0.2364
 0.2569  0.5000  0.2431
 0.2795  0.4904  0.2301
 0.2758  0.4787  0.2455
[torch.FloatTensor of size 5x3]



In [16]:
for epoch in range(300):
    for sent, tags in train_data:
        model.zero_grad()
        
        model.hidden = model.init_hidden()
        
        sent_in = prepare_seq(sent, word2idx)
        targets = prepare_seq(tags, tag2idx)
        
        tag_probs = model(sent_in)
        
        loss = loss_fun(tag_probs, targets)
        loss.backward()
        optimizer.step()
        

In [17]:
inputs = prepare_seq(train_data[0][0], word2idx)
tag_probs = model(inputs)

print(tag_probs)

Variable containing:
 0.8783  0.0736  0.0481
 0.0204  0.9511  0.0285
 0.4076  0.3669  0.2254
 0.7931  0.0899  0.1170
 0.0380  0.9145  0.0474
[torch.FloatTensor of size 5x3]



RuntimeError: max() received an invalid combination of arguments - got (axis=int, ), but expected one of:
 * (int dim, bool keepdim)
 * (Variable other)
      didn't match because some of the keywords were incorrect: axis
 * ()
      didn't match because some of the keywords were incorrect: axis
